### Automating Web Application Security Testing with OWASP ZAP and SQLMap

### Table of Contents

1. [Introduction](#introduction)
2. [Prerequisites and Setup](#prerequisites-and-setup)
   - [Tools Used](#tools-used)
   - [Installation Instructions](#installation-instructions)
   - [Setting Up a Test Environment](#setting-up-a-test-environment)
3. [OWASP ZAP: Automating Web Application Scans](#owasp-zap-automating-web-application-scans)
   - [Launching OWASP ZAP](#launching-owasp-zap)
   - [Using the ZAP API for Automated Scanning](#using-the-zap-api-for-automated-scanning)
   - [Analyzing ZAP Results](#analyzing-zap-results)
4. [SQLMap: Automating SQL Injection Testing](#sqlmap-automating-sql-injection-testing)
   - [Basic SQLMap Commands](#basic-sqlmap-commands)
   - [Automating SQLMap with Python](#automating-sqlmap-with-python)
   - [Parsing SQLMap Output](#parsing-sqlmap-output)
5. [Visualizing Vulnerability Data](#visualizing-vulnerability-data)
   - [Data Extraction from ZAP and SQLMap](#data-extraction-from-zap-and-sqlmap)
   - [Visualizing with Matplotlib and Seaborn](#visualizing-with-matplotlib-and-seaborn)
6. [Generating Interactive Dashboards](#generating-interactive-dashboards)
   - [Using Plotly for Interactive Visualizations](#using-plotly-for-interactive-visualizations)
   - [Creating a Vulnerability Dashboard](#creating-a-vulnerability-dashboard)
7. [Remediation and Retesting](#remediation-and-retesting)
   - [Tracking Vulnerability Fixes](#tracking-vulnerability-fixes)
   - [Running Retests to Verify Fixes](#running-retests-to-verify-fixes)
8. [Conclusion](#conclusion)



#### Automating Web Application Security Testing with OWASP ZAP and SQLMap

##### Introduction

In this notebook, we will demonstrate how to automate the process of web application security testing using two popular open-source tools, **OWASP ZAP** and **SQLMap**. OWASP ZAP helps in detecting security vulnerabilities in web applications such as Cross-Site Scripting (XSS), broken authentication, and other web-based issues, while SQLMap specializes in detecting and exploiting SQL injection vulnerabilities. 

The goal of this project is to automate the scanning process, extract and visualize vulnerabilities, and provide insights into how these vulnerabilities can be mitigated. We will leverage Python to automate tool interactions and visualize vulnerability data with libraries like **Matplotlib**, **Seaborn**, and **Plotly** for advanced dashboards.


#### 2. Prerequisites and Setup

Before diving into automated web application scanning, we need to ensure that the required tools are installed and configured properly. This section will walk through the installation process for OWASP ZAP and SQLMap, setting up a testing environment, and the basic system requirements.

### Tools Used

We will use the following tools:
- **OWASP ZAP**: An open-source web application security scanner.
- **SQLMap**: An open-source tool for automating SQL injection testing.
- **Python**: The programming language we will use to automate the interactions with ZAP and SQLMap.
- **Matplotlib, Seaborn, Plotly**: Python libraries for visualizing data.

### Installation Instructions

1. **Installing OWASP ZAP**
   - Download OWASP ZAP from the [official website](https://www.zaproxy.org/download/).
   - Follow the installation instructions for your operating system.
   - Verify installation by running the following command in your terminal:

This will confirm that ZAP is installed correctly.

- **Installing SQLMap**
   - Clone the SQLMap GitHub repository:

In [ ]:
git clone https://github.com/sqlmapproject/sqlmap.git

 - Navigate into the `sqlmap` directory and run it:

In [ ]:
cd sqlmap
python sqlmap.py --help

This should print the SQLMap help documentation, confirming that the tool is installed properly.

- **Setting up Python Environment**
   - Create a Python virtual environment and install the necessary libraries:

In [ ]:
python -m venv venv

# source venv/bin/activate  # For Linux/MacOS

# On Windows: venv\Scripts\activate

pip install matplotlib seaborn plotly requests

### Setting Up a Test Environment

For testing purposes, it's useful to use a vulnerable web application like **DVWA (Damn Vulnerable Web Application)** or **Mutillidae**:
   - Download and set up [DVWA](https://github.com/digininja/DVWA) or [Mutillidae](https://github.com/webpwnized/mutillidae).
   - Make sure your test environment is running so OWASP ZAP and SQLMap can target it.

#### 3. OWASP ZAP: Automating Web Application Scans

#### Launching OWASP ZAP

You can launch OWASP ZAP either via the GUI or in **headless mode**. For automation purposes, we will use the **ZAP API** to interact programmatically with the tool.

1. Start OWASP ZAP in headless mode using the following command:

In [ ]:
 zap.sh -daemon -port 8090 -host 127.0.0.1

This starts ZAP as a background process, listening on port 8090.

- Verify that ZAP is running by opening your browser and navigating to:

In [ ]:
http://localhost:8090

##### Using the ZAP API for Automated Scanning
ZAP exposes an API for automating scans. First, install the python-owasp-zap-v2.4 package to interact with the API:

In [ ]:
pip install python-owasp-zap-v2.4

The following Python code will automate the process of scanning a target web app:

In [ ]:
from zapv2 import ZAPv2
import time

# Connect to ZAP
zap = ZAPv2(apikey='your_api_key', proxies={'http': 'http://127.0.0.1:8090', 'https': 'http://127.0.0.1:8090'})

# Define the target
target_url = 'http://your-test-app-url.com'

# Start a new session
zap.core.new_session(name='test_session', overwrite=True)

# Access the target website
print(f"Accessing target {target_url}")
zap.urlopen(target_url)

# Perform passive scan
print(f"Starting passive scan on {target_url}")
time.sleep(2)  # Give some time for ZAP to analyze the site

# Start active scan
print(f"Starting active scan on {target_url}")
scan_id = zap.ascan.scan(target_url)

# Monitor the status of the scan
while int(zap.ascan.status(scan_id)) < 100:
    print(f"Scan progress: {zap.ascan.status(scan_id)}%")
    time.sleep(5)

print(f"Scan completed for {target_url}")

#### Analyzing ZAP Results
After the scan completes, the results can be retrieved and analyzed. Here's how to extract the discovered vulnerabilities:

In [ ]:
alerts = zap.core.alerts(baseurl=target_url)
for alert in alerts:
    print(f"Alert: {alert['alert']}, Risk: {alert['risk']}, URL: {alert['url']}")


You can save the results to a JSON file for further analysis and visualization:

In [ ]:
import json

with open('zap_scan_results.json', 'w') as f:
    json.dump(alerts, f, indent=4)


### Summary:
This notebook demonstrates how to automate the process of web application security testing using two widely known tools, OWASP ZAP and SQLMap. The Jupyter notebooks guide users through setting up automated vulnerability scanning workflows, performing web security scans, and detecting critical issues like SQL injection and cross-site scripting (XSS). Python is used to integrate with the ZAP API and automate SQLMap commands, while popular data visualization libraries such as Matplotlib, Seaborn, and Plotly are leveraged to analyze and visualize the vulnerabilities found. The project also provides insights into tracking and verifying the remediation of security flaws.